# Using Pypot for low-level communication with dynamixel motors

You will need to use Python >= 2.7 or >= 3.4. 

It works on Mac/Win/Linux but serial drivers tends to be better supported on Unix system.

## Installation

- via pip: ```pip install pypot```
- via the source if you prefer: https://github.com/poppy-project/pypot/

Make sure you are using a recent version >3.1!

In [2]:
import pypot

print(pypot.__version__)

3.1.1


## Dynamixel protocol

Two different kinds of IO:
- DxlIO
- Dxl320IO

- The first one is for protocol one (AX, MX, RX motors). It's the one used by Reachy.
- The second one is used for XL320 motors and protocol 2. 

Different protocol but basically the same idea behind.

### Some examples using protocol 1

In [4]:
import pypot.dynamixel.protocol.v1 as protocol

In [5]:
protocol.DxlPingPacket?

All motors should have a unique id. The idea is a number from 1 to 253. Don't go out range!

In [6]:
packet = protocol.DxlPingPacket(23)

In [14]:
[int(b) for b in packet.to_array()]

[255, 255, 23, 2, 1, 229]

In more details:

- [255, 255, 23, 2] is the header (target id=23, data length=2)
- 1 is the instruction code (PING)
- 229 is the CRC

In [19]:
packet = protocol.DxlReadDataPacket(id=2, address=0x24, length=2)
[int(b) for b in packet.to_array()]

[255, 255, 2, 4, 2, 36, 2, 209]

In more details:
- [255, 255, 2, 4] is the header (target id=2, data length=4)
- 2 is the instruction code (READ_DATA)
- 36 is the register address
- 2 is the register length
- 209 is the CRC

Motors answer with similar kind of packet. They are called StatusPacket.

Not so easy to understand :-)

## Connecting to a motor bus

In [21]:
from pypot.dynamixel import DxlIO

DxlIO wraps a USB to serial communication with dynamixel protocol. Reachy motors are configured to use 1M bauds.

In [22]:
io = DxlIO(port='...', baudrate=1000000)

You need to find the name of the serial port you will use. It depends on your OS. On windows you may also have to install a specific driver for the dongle we are using: USB2AX. On Mac/Linux it should directly work.

## Knowing your id

You first need to find your motor id. This is mandataroy as in the dynamixel protocol all messages must have a target id.

You can use the scan method to search for id. It will basically ping the whole range of ids and wait for answer. It can take a bit of time, as it waits for timeout.

In [ ]:
ids = io.scan()
print(ids)

If you know the range of id, you can make it faster.

In [ ]:
ids = io.scan(range(10, 20))
print(ids)

*Be careful, if you send a message to an id that does not correspond to a motor, pypot will actually wait for the timeout to fire and you will get a timeout exception. This can really slow down your communication with the motors.*

**Never uses motors with the same id on a single bus. Both motors will try to answer at about the same time resulting in collision and inpredictible communication.**

## Reading values

Now that you know the id, you can reading for a motor register. Instead of building yourself the ReadData Packet, pypot handles that for you. You actually don't need to know the address or length of register, but simply its name.

In [ ]:
io.get_present_position(ids)

To speedup communication, pypot uses list of ids instead of a single id. In the background it uses a more complex instruction that reads the same regsiter for all motors. It's faster than asking them one by one.

You can still read the register of a single motor.

In [ ]:
io.get_present_position([21])

Motor have a lot of different regsiters. You can find them in the documentation: http://poppy-project.github.io/pypot/pypot.dynamixel.html#module-pypot.dynamixel.io or via code completion.

In [ ]:
io.get_

The most useful are:
- present_position
- moving_speed
- present_load
- present_temperature
- is_moving

You will find the documentation of all specific regsiters directly on Robotis dynamixel support website.

You also may have noticed that pypot use metrics system as much as possible. Position are in degree, speed in degree per second, etc. This conversion from dynamixel values is automatically done when running and ```io.get_*``` command. It can also be disabled via:

In [ ]:
io.get_present_position(ids, convert=False)

You can also ask for the model of a motor. This can be particularly useful as they have slighltly different internal constant (e.g. speed max).

In [ ]:
io.get_model(ids)

Finally, you can also read multiple registers at the same time to speed up communication. It's only working when the registers are contiguous.

In [ ]:
io.get_present_position_speed_load(ids)

## Writing values

In the same way you saw how to read specific register of motors, you can also write new values to register.

When readings values you specify a list of ids, to set new values you specify a python dictionnary with id as key and the new value you want to set as value:

```
d = {
   id_1: val_1,
   id_2: val_2,
   ...
   id_n: val_n
  }
```

## Stiff vs Compliant mode

But before being able to send values and make motor moves, there is one still important step. When plugged the motors are compliant by default. This means that they can freely be moved by hand. 

You can test if your motor are compliant by tring to move them by hand. You can also observe that their *present_position* is still actually changing when moved by hand.

Yet if you want to make them turn, you need to put them in stiff mode. To do that in pypot, you use a special command *enable_torque*.

In [ ]:
io.enable_torque(ids)

Now you can try again to move them by hand, it should not be possible anymore.

To change the motor position you can use the *set_goal_position* method.

In [ ]:
# This will move  
# the motor 2 to 0.0 degree 
# and
# the motor 3 to 45.0 degree.

io.set_goal_position({
    2: 0.0,
    3: 45.0,
})

**Note**: Maybe you noticed that to access the position we have used *present_position* and here we are using *goal_position*. This is a very important distinction for dynamixel motors.

* The present_position always represents the current motor position, whenever it's moving or not. This by definition can not be set but only read.
* The goal_position is the motor target position, meaning the position it will try to reach. This can be set to define a new target for the motor but also read for instance to measure difference between the target and the real position.

Whether or not the present and goal position are the same (or close enough) depends of the motor load, the pid, if the goal position is actually reachable (inside the angle limits), etc.

Some of those parameters, like the pid, can be modify using pypot as well (but use it with caution as bad values for pid can really quickly damage your motor!).

### Moving speed or maximum speed

You can modify the maximum speed that a motor can reach via *set_moving_speed* method. For instance:

In [ ]:
io.set_moving_speed({
    2: 50.0,
    3: 50.0,
})

This command will limit the speed of motor 2 and 3 to 50 degrees per second. Depending on their model, dynamixel motors can reach up to 700 degrees per second.

**It's important to note that this limit is not a physical one, depending on the pid parameters you may temporally exceed this maximum speed. But most of the time the motor will keep its speed below this value.**

Now try to make the motor move again and you should be able to see the difference of speed.

You now have all tools needed to make your motor follow a trajectory, like for instance a Sinus.

The time needed to communicate with the motors will depend on your USB2serial driver and on the number of motors plugged to the bus. Yet, a rule of thumb is to try to reach a 50Hz control. Higher control rate is actually useless as the position servoing will not react fast enough and slower control rate may result in shaky motion.

### Torque

Among other important registers, you can also modify the torque limit of a motor via *set_torque_limit*. Here again, the motor will try to respect this consign but may exceed it.

In [ ]:
io.set_torque_limit({
    3: 25.0 # expressed in % of the real maximum torque of the motor
})

You can also disable torque entirely to switch to compliant mode again.

In [ ]:
io.disable_torque(ids)

## EEPROM and RAM

*Note that some register are in the EEPROM part of the motor micro-controller and will be preserved even when un-plugged (such as pid or angle limits). Other like goal_position or compliancy will be reset as soon as they are unplugged.*

You can find out if a regsiter is in the EEPROM or not in Robotis Dynamixel documentation.

## Configure motors utility

Pypot also comes with the command line utility tool that you used to configure the motors for Reachy.